In [1]:
import branca
import csv
import datetime
import folium
import sqlite3
from sqlalchemy import create_engine
import ipywidgets as widgets
import numpy as np
import pandas as pd
import draw_markers
import table_builder


In [2]:

with open('data/radius.csv',encoding='ascii',errors='ignore') as csvfile:
    radius = pd.read_csv(csvfile)

In [3]:
def normalize_BDA_count(dataset, num_days):
    build_end_dt = dataset.build_end_dt.apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    build_start_dt = dataset.build_start_dt.apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    coll_dt = dataset.coll_dt.apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    
    coll_after = ((coll_dt - build_end_dt)/np.timedelta64(1,'D')).astype(int)
    coll_before =((coll_dt - build_start_dt)/np.timedelta64(1,'D')).astype(int)
    coll_during = np.where(((coll_after<0)&(coll_before>0)),0,1)

    coll_days_from_build = np.zeros(len(build_end_dt))

    for i in range(0,len(build_end_dt)):
        if coll_during[i] == 0:
            coll_days_from_build[i] = 0
        elif coll_after[i] > 0:
            coll_days_from_build[i] = coll_after[i]
        else:
            coll_days_from_build[i] = coll_before[i]

    raw_coll_before = np.where(((build_start_dt > coll_dt) & (coll_dt > (build_start_dt - datetime.timedelta(days = num_days)))), 1, 0)
    raw_coll_after = np.where(((build_end_dt < coll_dt) & (coll_dt < (build_end_dt + datetime.timedelta(days = num_days)))), 1, 0)
    raw_coll_during = np.where(((build_end_dt >= coll_dt) & (build_start_dt <= coll_dt)), 1, 0)
    days_during = (build_end_dt-build_start_dt)
 
    days_during = (days_during/np.timedelta64(1,'D')).astype(int) 
    adjustment_factor = num_days/days_during
    adjusted_coll_during = raw_coll_during * adjustment_factor
    
    dataset['coll_before'] = raw_coll_before
    dataset['coll_during'] = adjusted_coll_during
    dataset['coll_after'] = raw_coll_after
    dataset['coll_days_from_build'] = coll_days_from_build
    dataset = dataset.iloc[:,1:]
    return dataset

Coll_Within_One_Year_of_Const =  normalize_BDA_count(radius, 365)
# Coll_Within_One_Year_of_Const.to_csv('data/Coll_Within_One_Year_of_Const.csv')

In [4]:
Coll_Within_One_Year_of_Const

,build_end_dt,build_id,build_lat,build_long,build_start_dt,coll_dt,coll_id,coll_lat,coll_long,radius,coll_before,coll_during,coll_after,coll_days_from_build
0,2017-08-24,6401146,47.632094,-122.341914,2015-04-24,2013-03-26,162244723,47.628446,-122.342315,1334.548944,0,0.000000,0,-759.0
1,2017-08-24,6401146,47.632094,-122.341914,2015-04-24,2013-04-02,162245024,47.633693,-122.342283,590.318832,0,0.000000,0,-752.0
2,2017-08-24,6401146,47.632094,-122.341914,2015-04-24,2016-03-27,162246458,47.632299,-122.344886,736.677075,0,0.427902,0,0.0
3,2017-08-24,6401146,47.632094,-122.341914,2015-04-24,2014-12-20,162246547,47.630997,-122.342292,410.916863,1,0.000000,0,-125.0
4,2017-08-24,6401146,47.632094,-122.341914,2015-04-24,2016-06-23,162246563,47.632300,-122.342281,117.684655,0,0.427902,0,0.0
5,2017-08-24,6401146,47.632094,-122.341914,2015-04-24,2013-04-19,162371491,47.632411,-122.340922,270.469861,0,0.000000,0,-735.0
6,2017-08-24,6401146,47.632094,-122.341914,2015-04-24,2013-04-20,162371644,47.634733,-122.343472,1036.534100,0,0.000000,0,-734.0
7,2017-08-24,6401146,47.632094,-122.341914,2015-04-24,2013-01-14,162371934,47.634733,-122.343472,1036.534100,0,0.000000,0,-830.0
8,2017-08-24,6401146,47.632094,-122.341914,2015-04-24,2013-03-08,162373361,47.628278,-122.340419,1439.965767,0,0.000000,0,-777.0
9,2017-08-24,6401146,47.632094,-122.341914,2015-04-24,2013-01-18,162373583,47.629309,-122.343532,1091.567953,0,0.000000,0,-826.0
